In [1]:
#importing the libraries
import requests 
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import numpy as np

In [2]:
#generating all webpages of each county
x = ['https://www.redfin.com/county/303/CA/Alameda-County',
     'https://www.redfin.com/county/304/CA/Alpine-County',
     'https://www.redfin.com/county/306/CA/Butte-County',
     'https://www.redfin.com/county/307/CA/Calaveras-County',
     'https://www.redfin.com/county/311/CA/El-Dorado-County',
     'https://www.redfin.com/county/312/CA/Fresno-County',
     'https://www.redfin.com/county/324/CA/Mariposa-County',
     'https://www.redfin.com/county/329/CA/Monterey-County',
     'https://www.redfin.com/county/330/CA/Napa-County',
     'https://www.redfin.com/county/331/CA/Nevada-County',
     'https://www.redfin.com/county/340/CA/San-Francisco-County',
     'https://www.redfin.com/county/345/CA/Santa-Clara-County',
     'https://www.redfin.com/county/346/CA/Santa-Cruz-County',
     'https://www.redfin.com/county/359/CA/Yolo-County']

county_links =[]
for c_l in x:
    c_text = requests.get(c_l,headers={'User-Agent': 'Mozilla/5.0'}).text
    c_soup = BS(c_text,'html5lib')
    page_number=int(c_soup.find('span',attrs={'class':'pageText'}).text.split('of ')[1])
    county_links.append(c_l)
    if page_number>1:
        for num in range(2,page_number+1):
            str_num=str(num)
            new_c_l = c_l+'/page-'+str_num
            county_links.append(new_c_l)
county_links

['https://www.redfin.com/county/303/CA/Alameda-County',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-2',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-3',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-4',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-5',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-6',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-7',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-8',
 'https://www.redfin.com/county/303/CA/Alameda-County/page-9',
 'https://www.redfin.com/county/304/CA/Alpine-County',
 'https://www.redfin.com/county/304/CA/Alpine-County/page-2',
 'https://www.redfin.com/county/306/CA/Butte-County',
 'https://www.redfin.com/county/306/CA/Butte-County/page-2',
 'https://www.redfin.com/county/306/CA/Butte-County/page-3',
 'https://www.redfin.com/county/306/CA/Butte-County/page-4',
 'https://www.redfin.com/county/306/CA/Butte-County/page-5',
 'https://www.redfin.com/

In [3]:
#Creating a dictionary with key - county , value - List of properties
#extracting all links in a webpage using a href tags and using regex to identify the links for properties
Property_set={}

for url in county_links:
    county_url = url
    county_name_list = re.findall('CA/([^/]+)',county_url)
    text = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text
    soup = BS(text,'html5lib')
    all_links = [a['href'] for a in soup('a') if a.has_attr('href')] #extracting all links on the webpage
    regex = r"^/CA" #extracting only links for properties listed
    one_county_prop_url = [link for link in all_links if re.match(regex,link)]
    for a in range(0, len(one_county_prop_url)):
        one_county_prop_url[a]='https://www.redfin.com'+one_county_prop_url[a]
    final_url =[]
    for l in one_county_prop_url:
        if l not in final_url:
            final_url.append(l)
    for x in county_name_list:
        if x in Property_set:
            Property_set[x]=Property_set[x]+final_url
        else:
            Property_set[x]=final_url
        
Property_set

{'Alameda-County': ['https://www.redfin.com/CA/Oakland/655-12th-St-94607/unit-112/home/1553948',
  'https://www.redfin.com/CA/Oakland/4475-Redding-St-94619/home/561724',
  'https://www.redfin.com/CA/Oakland/6501-San-Pablo-Ave-94608/unit-404/home/12116522',
  'https://www.redfin.com/CA/Hayward/29467-Taylor-Ave-94544/home/826626',
  'https://www.redfin.com/CA/San-Leandro/882-Tex-CT-94577/home/184536086',
  'https://www.redfin.com/CA/San-Leandro/1258-144th-Ave-94578/home/1731777',
  'https://www.redfin.com/CA/Oakland/4000-Altamont-Ave-94605/home/563941',
  'https://www.redfin.com/CA/Oakland/2215-88th-Ave-94605/home/1543860',
  'https://www.redfin.com/CA/Hayward/28860-Rochelle-Ave-94544/home/1426663',
  'https://www.redfin.com/CA/Fremont/38888-Salmon-Ter-94536/home/1072155',
  'https://www.redfin.com/CA/San-Lorenzo/17393-Via-Magdalena-94580/home/1861754',
  'https://www.redfin.com/CA/Oakland/725-E-21st-St-94606/home/533532',
  'https://www.redfin.com/CA/Hayward/325-Valle-Vista-Ave-94544/un

In [4]:
#creating lists for the attributes and extracting property information using tags like div, span, etc
#using try & except blocks to identify errors and handle them

Street= []
City = []
State = []
Zipcode = []
Price = []
Beds = []
Bath = []
Sqft = []
Property_Type = []
Year_Built = []
Status = []
Walk_Score = []
Bike_Score = []
Acre=[]
count=0
for county in Property_set:
    for prop in Property_set[county]:
        print(count)
        count=count+1
        al_url = prop
        al_text = requests.get(al_url,headers={'User-Agent': 'Mozilla/5.0'}).text
        al_soup = BS(al_text,'html5lib')
        
        try:
            street = al_soup.find('div',attrs={'class':'street-address'}).text.split(',')[0]
        except:
            street=np.nan
            print(al_url)
            print('street')
        Street.append(street)
        
        try:
            city = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[0]
        except:
            city=np.nan
            print(al_url)
            print('city')
        City.append(city)
        
        try:
            state = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[1].split(' ')[0]
        except:
            state=np.nan
            print(al_url)
            print('state')
        State.append(state)
        
        try:
            zipcode = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[-1].split(' ')[-1]
            
        except:
            zipcode=np.nan
            print(al_url)
            print('zipcode')
        Zipcode.append(zipcode)    
        
        try:
            price = al_soup.find('div',attrs={'class':'statsValue'}).text.split('$')[1]
        except:
            price=np.nan
            print(al_url)
            print('price')
        Price.append(price)
        
        try:
            bed_str = al_soup.find('div',attrs={'data-rf-test-id':'abp-beds'}).text.split('B')[0]
            beds = int(bed_str)
        except:
            beds=np.nan
            print(al_url)
            print('beds')
        Beds.append(beds)
        
        try:
            bath_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-baths'}).text.split('B')[0]
            bath = float(bath_str)
        except:
            bath=np.nan
            print(al_url)
            print('bath')
        Bath.append(bath)
        
        try:
            sqft_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-sqFt'}).text.split('S')[0].replace(',','')
            sqft = float(sqft_str)
        except:
            sqft=np.nan
            print(al_url)
            print('sqft')
        Sqft.append(sqft)
        
        try:
            acre_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-sqFt'}).text.split('A')[0]
            acre = float(acre_str)
        except:
            acre=np.nan
            print(al_url)
            print('acre')
        Acre.append(acre)
        
        
        try:
            div_prop = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            property_type =''
            for tag in div_prop:
                if(tag.span.text=='Property Type'):
                    property_type = property_type+tag.find('span',attrs={'class':'content text-right'}).text
        except:
            property_type = np.nan
            print(al_url)
            print('prop_type')
        Property_Type.append(property_type)
        
        try:
            div_year_built = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            year_built =''
            for tag in div_year_built:
                if(tag.span.text=='Year Built'):
                    year_built = int(year_built+tag.find('span',attrs={'class':'content text-right'}).text)
        except:
            year_built=np.nan
            print(al_url)
            print('year')
        Year_Built.append(year_built)
        
        try:
            div_status = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            status =''
            for tag in div_status:
                if(tag.span.text=='Status'):
                    status = tag.find('span',attrs={'class':'DefinitionFlyoutLink inline-block underline clickable'})
                    if status == None:
                        status = tag.find('span',attrs={'class':'content text-right'})
        except:
            status=np.nan
            print(al_url)
            print('status')
        Status.append(status.text)

        try:
            walk_score = int(al_soup.find('div',attrs={'class':'transport-icon-and-percentage walkscore'}).text.split(' /')[0])
        except:
            walk_score=np.nan
            print(al_url)
            print('walk')
        Walk_Score.append(walk_score)
        
        try:
            bike_score = int(al_soup.find('div',attrs={'class':'transport-icon-and-percentage bikescore'}).text.split(' /')[0])
        except:
            bike_score=np.nan
            print(al_url)
            print('bike')
        Bike_Score.append(bike_score)

0
https://www.redfin.com/CA/Oakland/655-12th-St-94607/unit-112/home/1553948
acre
1
https://www.redfin.com/CA/Oakland/4475-Redding-St-94619/home/561724
acre
2
https://www.redfin.com/CA/Oakland/6501-San-Pablo-Ave-94608/unit-404/home/12116522
acre
3
https://www.redfin.com/CA/Hayward/29467-Taylor-Ave-94544/home/826626
acre
4
https://www.redfin.com/CA/San-Leandro/882-Tex-CT-94577/home/184536086
acre
5
https://www.redfin.com/CA/San-Leandro/1258-144th-Ave-94578/home/1731777
acre
6
https://www.redfin.com/CA/Oakland/4000-Altamont-Ave-94605/home/563941
acre
7
https://www.redfin.com/CA/Oakland/2215-88th-Ave-94605/home/1543860
acre
8
https://www.redfin.com/CA/Hayward/28860-Rochelle-Ave-94544/home/1426663
acre
9
https://www.redfin.com/CA/Fremont/38888-Salmon-Ter-94536/home/1072155
acre
10
https://www.redfin.com/CA/San-Lorenzo/17393-Via-Magdalena-94580/home/1861754
acre
11
https://www.redfin.com/CA/Oakland/725-E-21st-St-94606/home/533532
acre
12
https://www.redfin.com/CA/Hayward/325-Valle-Vista-Ave-

https://www.redfin.com/CA/Fremont/4725-Balboa-Way-94536/home/1534030
acre
104
https://www.redfin.com/CA/Pleasanton/6120-Sanctuary-Ln-94566/home/17519332
acre
105
https://www.redfin.com/CA/Oakland/1677-12th-St-94607/home/184471981
acre
106
https://www.redfin.com/CA/Newark/6195-Marguerite-Dr-94560/home/869134
acre
107
https://www.redfin.com/CA/Oakland/2446-24th-Ave-94601/home/1062911
acre
108
https://www.redfin.com/CA/Pleasanton/1904-Zenato-Pl-94566/home/1739532
acre
109
https://www.redfin.com/CA/Oakland/546-Glen-View-Ave-94610/home/182535482
acre
110
https://www.redfin.com/CA/Oakland/485-8th-St-94607/unit-503/home/184468148
acre
111
https://www.redfin.com/CA/Fremont/4916-Cody-Ct-94538/home/1325790
acre
112
https://www.redfin.com/CA/Oakland/544-Glenview-Ave-94610/home/530610
acre
113
https://www.redfin.com/CA/Berkeley/2271-Virginia-St-94709/home/529251
acre
114
https://www.redfin.com/CA/Newark/6323-Central-Ave-94560/home/1470919
acre
115
https://www.redfin.com/CA/Piedmont/771-Kingston-Av

https://www.redfin.com/CA/Livermore/1610-El-Dorado-Ct-94550/home/2022655
acre
189
https://www.redfin.com/CA/Oakland/4080-Kuhnle-Ave-94605/home/564479
acre
190
https://www.redfin.com/CA/Albany/555-Pierce-St-94706/unit-1321/home/1845503
acre
191
https://www.redfin.com/CA/Oakland/2441-E-22nd-St-94601/home/880140
acre
192
https://www.redfin.com/CA/Piedmont/32-Bellevue-Ave-94611/home/1378759
acre
193
https://www.redfin.com/CA/Pleasanton/968970-Happy-Valley-Rd-94566/home/184411681
beds
https://www.redfin.com/CA/Pleasanton/968970-Happy-Valley-Rd-94566/home/184411681
bath
https://www.redfin.com/CA/Pleasanton/968970-Happy-Valley-Rd-94566/home/184411681
sqft
194
https://www.redfin.com/CA/Pleasanton/3003-Briggs-Ct-94588/home/810377
acre
195
https://www.redfin.com/CA/Oakland/5225-Cochrane-Ave-94618/home/1712983
acre
196
https://www.redfin.com/CA/Hayward/641-Minerva-St-94544/home/1022704
acre
197
https://www.redfin.com/CA/Hayward/4019-Lambert-Way-94541/home/143119671
acre
198
https://www.redfin.com

https://www.redfin.com/CA/San-Leandro/342-Marina-Blvd-94577/home/967251
acre
279
https://www.redfin.com/CA/Hayward/544-Marlin-Ct-94544/home/1171866
acre
280
https://www.redfin.com/CA/Berkeley/1903-Bonita-Ave-94704/unit-3/home/2038930
acre
281
https://www.redfin.com/CA/Fremont/43678-Skye-Rd-94539/home/1637975
acre
282
https://www.redfin.com/CA/Union-City/3995-Smith-St-94587/home/184319344
beds
https://www.redfin.com/CA/Union-City/3995-Smith-St-94587/home/184319344
bath
https://www.redfin.com/CA/Union-City/3995-Smith-St-94587/home/184319344
sqft
https://www.redfin.com/CA/Union-City/3995-Smith-St-94587/home/184319344
acre
283
https://www.redfin.com/CA/San-Leandro/576-Begier-Ave-94577/home/1834218
acre
284
https://www.redfin.com/CA/Oakland/2142-64th-Ave-94621/home/1330130
acre
285
https://www.redfin.com/CA/Union-City/35447-Monterra-Cir-94587/home/1631684
acre
286
https://www.redfin.com/CA/Oakland/Sheridan-Rd-94603/home/22694554
beds
https://www.redfin.com/CA/Oakland/Sheridan-Rd-94603/home/

https://www.redfin.com/CA/Bear-Valley/225-Bear-Valley-Rd-95223/home/184445593
acre
356
https://www.redfin.com/CA/Kirkwood/1399-Kirkwood-Meadows-Dr-95646/unit-326-8/home/183799548
acre
357
https://www.redfin.com/CA/Kirkwood/13-Palisades-Dr-95646/home/175107922
beds
https://www.redfin.com/CA/Kirkwood/13-Palisades-Dr-95646/home/175107922
bath
https://www.redfin.com/CA/Kirkwood/13-Palisades-Dr-95646/home/175107922
sqft
358
https://www.redfin.com/CA/Kirkwood/1050-Kirkwood-Meadows-DR-95646/unit-5IVB/home/183816701
acre
359
https://www.redfin.com/CA/Kirkwood/1399-Kirkwood-Meadows-Dr-95646/unit-205-7/home/184535655
acre
360
https://www.redfin.com/CA/Markleeville/88-Pioneer-Trl-96120/unit-304/home/183727638
beds
https://www.redfin.com/CA/Markleeville/88-Pioneer-Trl-96120/unit-304/home/183727638
bath
https://www.redfin.com/CA/Markleeville/88-Pioneer-Trl-96120/unit-304/home/183727638
sqft
https://www.redfin.com/CA/Markleeville/88-Pioneer-Trl-96120/unit-304/home/183727638
bike
361
https://www.redf

...

https://www.redfin.com/CA/Watsonville/1759-Santa-Victoria-Ave-95076/home/167260064
beds
https://www.redfin.com/CA/Watsonville/1759-Santa-Victoria-Ave-95076/home/167260064
bath
https://www.redfin.com/CA/Watsonville/1759-Santa-Victoria-Ave-95076/home/167260064
acre
4475
https://www.redfin.com/CA/Felton/9485-Zayante-Dr-95018/home/2368303
beds
https://www.redfin.com/CA/Felton/9485-Zayante-Dr-95018/home/2368303
bath
https://www.redfin.com/CA/Felton/9485-Zayante-Dr-95018/home/2368303
sqft
4476
https://www.redfin.com/CA/Ben-Lomond/300-Sweetwater-Ln-95005/home/2379122
beds
https://www.redfin.com/CA/Ben-Lomond/300-Sweetwater-Ln-95005/home/2379122
bath
https://www.redfin.com/CA/Ben-Lomond/300-Sweetwater-Ln-95005/home/2379122
sqft
4477
https://www.redfin.com/CA/Felton/11507-Lakeview-Ave-95018/home/23020901
beds
https://www.redfin.com/CA/Felton/11507-Lakeview-Ave-95018/home/23020901
bath
https://www.redfin.com/CA/Felton/11507-Lakeview-Ave-95018/home/23020901
sqft
https://www.redfin.com/CA/Felton/1

https://www.redfin.com/CA/West-Sacramento/3303-San-Vicente-Rd-95691/home/19485743
acre
4536
https://www.redfin.com/CA/Davis/2306-Cezanne-Ct-95618/home/19471924
acre
4537
https://www.redfin.com/CA/West-Sacramento/3309-Falcon-Ct-95691/home/167319005
acre
4538
https://www.redfin.com/CA/Davis/1314-Marina-Cir-95616/home/19466947
acre
4539
https://www.redfin.com/CA/Davis/4240-Arroyo-Ave-95618/home/19446606
acre
4540
https://www.redfin.com/CA/West-Sacramento/3973-Tule-St-95691/home/19041343
acre
4541
https://www.redfin.com/CA/West-Sacramento/3855-Tahoe-St-95691/home/19488191
acre
4542
https://www.redfin.com/CA/Woodland/1909-Miekle-Ave-95776/home/57314859
acre
4543
https://www.redfin.com/CA/West-Sacramento/3174-Longspur-Ln-95691/home/168693002
acre
4544
https://www.redfin.com/CA/Woodland/18-Clanton-Ave-95695/home/19477854
acre
4545
https://www.redfin.com/CA/Woodland/1216-Harley-Dr-95695/home/19455184
acre
4546
https://www.redfin.com/CA/Woodland/1734-Matmor-Rd-95776/home/184306999
acre
4547
htt

https://www.redfin.com/CA/Winters/0-Russell-BLVD-95694/home/183648069
beds
https://www.redfin.com/CA/Winters/0-Russell-BLVD-95694/home/183648069
bath
https://www.redfin.com/CA/Winters/0-Russell-BLVD-95694/home/183648069
sqft
4626
https://www.redfin.com/CA/Woodland/1958-Hawkins-Ct-95776/home/22616516
acre
4627
https://www.redfin.com/CA/Clarksburg/52100-Clarksburg-Rd-95612/home/183612816
beds
https://www.redfin.com/CA/Clarksburg/52100-Clarksburg-Rd-95612/home/183612816
bath
https://www.redfin.com/CA/Clarksburg/52100-Clarksburg-Rd-95612/home/183612816
sqft
4628
https://www.redfin.com/CA/Winters/1150-Ramos-Dr-95694/home/182371154
acre
4629
https://www.redfin.com/CA/Winters/1144-Ramos-Dr-95694/home/182371144
acre
4630
https://www.redfin.com/CA/Winters/1152-Ramos-Dr-95694/home/183325850
acre
4631
https://www.redfin.com/CA/Woodland/7-Leisureville-Cir-95776/home/183566058
acre
4632
https://www.redfin.com/CA/Woodland/2664-Ortiz-Ave-95776/home/19484829
acre
4633
https://www.redfin.com/CA/West-Sa

https://www.redfin.com/CA/Winters/0-California-128-95694/home/182050646
beds
https://www.redfin.com/CA/Winters/0-California-128-95694/home/182050646
bath
https://www.redfin.com/CA/Winters/0-California-128-95694/home/182050646
sqft
4690
https://www.redfin.com/CA/Capay/24004-County-Road-15B-95607/home/19457288
acre
4691
https://www.redfin.com/CA/Clarksburg/36313-N-Center-St-95612/home/19462289
acre
4692
https://www.redfin.com/CA/Davis/329-Cabrillo-Ave-95616/home/19469797
acre
4693
https://www.redfin.com/CA/West-Sacramento/Undisclosed-address-95605/home/19478685
acre
4694
https://www.redfin.com/CA/Sacramento/1461-Garden-Hwy-95833/home/22802246
acre
4695
https://www.redfin.com/CA/Sacramento/1457-Garden-Hwy-95833/home/22802245
acre
4696
https://www.redfin.com/CA/Woodland/2037-Smith-Dr-95776/home/182374762
acre
4697
https://www.redfin.com/CA/Brooks/18080-County-Road-70-95606/home/19457916
acre
4698
https://www.redfin.com/CA/Davis/945-Sparks-Ln-95616/home/144222030
acre
4699
https://www.redfi

https://www.redfin.com/CA/Winters/Shandalla/Residence-Two/home/182203634
acre
4749
https://www.redfin.com/CA/Winters/Shandalla/Residence-One/home/182203637
acre
4750
https://www.redfin.com/CA/Woodland/Iris/Residence-2013/home/182792273
acre
4751
https://www.redfin.com/CA/Woodland/Iris/Residence-1438/home/182792274
acre
4752
https://www.redfin.com/CA/Woodland/Iris/Residence-2031/home/182800925
acre
4753
https://www.redfin.com/CA/Woodland/Iris/Residence-2185/home/182862550
acre
4754
https://www.redfin.com/CA/Winters/The-Hideaway/The-Cottages-Residence-1/home/183203639
acre
4755
https://www.redfin.com/CA/Winters/The-Hideaway/The-Estates-Residence-3/home/183203640
acre
4756
https://www.redfin.com/CA/Winters/The-Hideaway/The-Classics-Residence-3/home/183203641
acre
4757
https://www.redfin.com/CA/Winters/The-Hideaway/The-Classics-Residence-4/home/183203643
acre
4758
https://www.redfin.com/CA/Winters/The-Hideaway/The-Classics-Residence-1/home/183203644
acre
4759
https://www.redfin.com/CA/Wood

In [5]:
County=[]
for key in Property_set:
    for value in Property_set[key]:
       County.append(key)
County

['Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-County',
 'Alameda-Co

In [6]:
###########################creating data frame ###################

In [7]:
df=pd.DataFrame(data={'Street':Street,
                      'City':City,
                      'County':County,
                      'State':State,
                      'Zipcode':Zipcode,
                      'Price':Price,
                      'WalkScore':Walk_Score,
                      'Bike_Score':Bike_Score,
                      'Beds':Beds,
                      'Bath':Bath,
                      'Sqft':Sqft,
                      'Acre':Acre,
                      'Property_Type':Property_Type,
                      'Year_Built':Year_Built,
                      'Status':Status})

In [8]:
df

,Street,City,County,State,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Acre,Property_Type,Year_Built,Status
0,655 12Th St #112,Oakland,Alameda-County,CA,94607,"649,000",96,91.0,2.0,2.0,1037.0,NaN,Condo,2003,New
1,4475 Redding St,Oakland,Alameda-County,CA,94619,"699,000",82,47.0,2.0,2.0,990.0,NaN,Single Family Residential,1930,New
2,6501 San Pablo Ave #404,Oakland,Alameda-County,CA,94608,"685,000",94,86.0,1.0,1.5,1010.0,NaN,Condo,2006,New
3,29467 Taylor Ave,HAYWARD,Alameda-County,CA,94544,"899,999",52,69.0,3.0,2.5,1469.0,NaN,Single Family Home,1989,Active
4,882 Tex Ct,San Leandro,Alameda-County,CA,94577,"909,640",70,68.0,4.0,3.5,1760.0,NaN,Townhouse,,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4775,Baltic,West Sacramento,Yolo-County,CA,95691,"752,655+",4,59.0,4.0,3.0,2770.0,NaN,Single Family Residential,,Active
4776,Argo,West Sacramento,Yolo-County,CA,95691,"733,925+",4,59.0,4.0,3.0,2105.0,NaN,Single Family Residential,,Active
4777,1158 Ramos Dr,Winters,Yolo-County,CA,95694,"631,255",4,35.0,NaN,NaN,NaN,NaN,Single Family ResidentialSingle Family Residen...,,Off Market
4778,1730 Matmor Rd,Woodland,Yolo-County,CA,95776,"518,187",25,34.0,NaN,NaN,NaN,NaN,Single Family ResidentialSingle Family Residen...,,Off Market


In [9]:
df.isna().sum()

Street              0
City                0
County              0
State               1
Zipcode             0
Price               1
WalkScore           0
Bike_Score        697
Beds             1479
Bath             1451
Sqft             1338
Acre             3524
Property_Type       0
Year_Built          2
Status              0
dtype: int64

In [10]:
################        Data cleaning        ####################

In [11]:
(df['Beds'].isna() & df['Bath'].isna()).sum()

1443

In [12]:
#If both beds and Bath have null values -> that indicates that the property is an empty land and there is no
#built house. We will drop such rows

In [13]:
df = df.dropna(subset=['Beds','Bath'],how='all')

In [14]:
#Street, City, State, Zipcode, Price have small number of null values. We will remove such rows since that will affect
#our analysis and insights

In [15]:
df=df.dropna(subset=['Street','City','State','Zipcode','Price','Year_Built'])

In [16]:
#replacing walk score and bike score null values with score of 0. Does not make sense to delete the entire row
#based on these scores as null value indicates score of 0.

In [17]:
df['WalkScore'].fillna(0,inplace=True)
df['Bike_Score'].fillna(0,inplace=True)

In [18]:
#We have removed properties where both beds and baths are 0
#To deal with remaining null values in Individual columns of Beds & Baths, we will replace it with 0 since 
#that means there is either no bedroom or not bathroom in that property

In [19]:
df['Beds'].fillna(0,inplace=True)
df['Bath'].fillna(0,inplace=True)

In [20]:
df.isna().sum()

Street              0
City                0
County              0
State               0
Zipcode             0
Price               0
WalkScore           0
Bike_Score          0
Beds                0
Bath                0
Sqft               41
Acre             3335
Property_Type       0
Year_Built          0
Status              0
dtype: int64

In [21]:
#deleting rows where sqft is not provided(null) since sqft is a part of our analysis

In [22]:
df=df.dropna(subset=['Sqft'])

In [23]:
#dropping the acre column. It was only used to identify the values where we do not have sqft

In [24]:
df=df.drop(columns=['Acre'])

In [25]:
df=df.reset_index()

In [26]:
df=df.drop(columns=['index'])

In [27]:
df

,Street,City,County,State,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,655 12Th St #112,Oakland,Alameda-County,CA,94607,"649,000",96,91.0,2.0,2.0,1037.0,Condo,2003,New
1,4475 Redding St,Oakland,Alameda-County,CA,94619,"699,000",82,47.0,2.0,2.0,990.0,Single Family Residential,1930,New
2,6501 San Pablo Ave #404,Oakland,Alameda-County,CA,94608,"685,000",94,86.0,1.0,1.5,1010.0,Condo,2006,New
3,29467 Taylor Ave,HAYWARD,Alameda-County,CA,94544,"899,999",52,69.0,3.0,2.5,1469.0,Single Family Home,1989,Active
4,882 Tex Ct,San Leandro,Alameda-County,CA,94577,"909,640",70,68.0,4.0,3.5,1760.0,Townhouse,,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,Residence 2134,Woodland,Yolo-County,CA,95776,"672,629+",25,34.0,3.0,3.0,2134.0,Single Family Residential,,Active
3290,Caspian,West Sacramento,Yolo-County,CA,95691,"787,690+",4,59.0,4.0,3.0,2869.0,Single Family Residential,,Active
3291,Dawson,West Sacramento,Yolo-County,CA,95691,"844,410+",4,59.0,4.0,3.5,3123.0,Single Family Residential,,Active
3292,Baltic,West Sacramento,Yolo-County,CA,95691,"752,655+",4,59.0,4.0,3.0,2770.0,Single Family Residential,,Active


In [28]:
#remove +sign & special characters
for i in range(0,len(df)):
    if(df['Price'][i][-1]=='+'):
        df['Price'][i]=df['Price'][i][0:-1]  
    else:
        df['Price'][i]=df['Price'][i]

/var/folders/lz/wgmsmvbx2_38594ymqn4b_f00000gn/T/ipykernel_68199/2455180488.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i]=df['Price'][i]
/var/folders/lz/wgmsmvbx2_38594ymqn4b_f00000gn/T/ipykernel_68199/2455180488.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i]=df['Price'][i][0:-1]


In [29]:
#####################         convert data type to relevant types     #####################

In [30]:
df['Price'] = df['Price'].str.replace(',','')

In [31]:
df['Price'] = df['Price'].astype(float)
df.loc[df['Year_Built'] == '', 'Year_Built'] = 0

In [32]:
df['Year_Built'] = df['Year_Built'].astype(int)

In [33]:
df.loc[df['Year_Built'] == 0, 'Year_Built'] = np.nan

In [34]:
df['WalkScore'] = df['WalkScore'].astype(int)

In [35]:
df.dtypes

Street            object
City              object
County            object
State             object
Zipcode           object
Price            float64
WalkScore          int64
Bike_Score       float64
Beds             float64
Bath             float64
Sqft             float64
Property_Type     object
Year_Built       float64
Status            object
dtype: object

In [36]:
#########################Normalization############################

In [37]:
df['Street']=df['Street'].apply(lambda x:x[0]+x[1:].lower())

In [38]:
df['City']=df['City'].apply(lambda x:x[0]+x[1:].lower())

In [39]:
df['State']=df['State'].apply(lambda x:x[0]+x[1:].lower())

In [40]:
df['Property_Type']=df['Property_Type'].apply(lambda x:x[0]+x[1:].lower())

In [41]:
df['Status']=df['Status'].apply(lambda x:x[0]+x[1:].lower())

In [42]:
df

,Street,City,County,State,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,655 12th st #112,Oakland,Alameda-County,Ca,94607,649000.0,96,91.0,2.0,2.0,1037.0,Condo,2003.0,New
1,4475 redding st,Oakland,Alameda-County,Ca,94619,699000.0,82,47.0,2.0,2.0,990.0,Single family residential,1930.0,New
2,6501 san pablo ave #404,Oakland,Alameda-County,Ca,94608,685000.0,94,86.0,1.0,1.5,1010.0,Condo,2006.0,New
3,29467 taylor ave,Hayward,Alameda-County,Ca,94544,899999.0,52,69.0,3.0,2.5,1469.0,Single family home,1989.0,Active
4,882 tex ct,San leandro,Alameda-County,Ca,94577,909640.0,70,68.0,4.0,3.5,1760.0,Townhouse,NaN,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,Residence 2134,Woodland,Yolo-County,Ca,95776,672629.0,25,34.0,3.0,3.0,2134.0,Single family residential,NaN,Active
3290,Caspian,West sacramento,Yolo-County,Ca,95691,787690.0,4,59.0,4.0,3.0,2869.0,Single family residential,NaN,Active
3291,Dawson,West sacramento,Yolo-County,Ca,95691,844410.0,4,59.0,4.0,3.5,3123.0,Single family residential,NaN,Active
3292,Baltic,West sacramento,Yolo-County,Ca,95691,752655.0,4,59.0,4.0,3.0,2770.0,Single family residential,NaN,Active


In [43]:
#########################removing duplicates######################

In [44]:
df.duplicated().value_counts()

False    3221
True       73
dtype: int64

In [45]:
df=df.drop_duplicates()

In [46]:
df

,Street,City,County,State,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,655 12th st #112,Oakland,Alameda-County,Ca,94607,649000.0,96,91.0,2.0,2.0,1037.0,Condo,2003.0,New
1,4475 redding st,Oakland,Alameda-County,Ca,94619,699000.0,82,47.0,2.0,2.0,990.0,Single family residential,1930.0,New
2,6501 san pablo ave #404,Oakland,Alameda-County,Ca,94608,685000.0,94,86.0,1.0,1.5,1010.0,Condo,2006.0,New
3,29467 taylor ave,Hayward,Alameda-County,Ca,94544,899999.0,52,69.0,3.0,2.5,1469.0,Single family home,1989.0,Active
4,882 tex ct,San leandro,Alameda-County,Ca,94577,909640.0,70,68.0,4.0,3.5,1760.0,Townhouse,NaN,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,Residence 2134,Woodland,Yolo-County,Ca,95776,672629.0,25,34.0,3.0,3.0,2134.0,Single family residential,NaN,Active
3290,Caspian,West sacramento,Yolo-County,Ca,95691,787690.0,4,59.0,4.0,3.0,2869.0,Single family residential,NaN,Active
3291,Dawson,West sacramento,Yolo-County,Ca,95691,844410.0,4,59.0,4.0,3.5,3123.0,Single family residential,NaN,Active
3292,Baltic,West sacramento,Yolo-County,Ca,95691,752655.0,4,59.0,4.0,3.0,2770.0,Single family residential,NaN,Active


In [47]:
df.kurt(numeric_only=True)

Price          63.106339
WalkScore      -1.197547
Bike_Score     -1.123569
Beds          407.009928
Bath          241.546424
Sqft          827.540087
Year_Built      0.130083
dtype: float64

In [48]:
df['Beds'].describe()

count    3221.000000
mean        3.252406
std         1.870147
min         0.000000
25%         2.000000
50%         3.000000
75%         4.000000
max        64.000000
Name: Beds, dtype: float64

In [49]:
df['Price'].describe()

count    3.221000e+03
mean     1.357104e+06
std      2.104088e+06
min      5.000000e+01
25%      4.800000e+05
50%      7.990000e+05
75%      1.475000e+06
max      3.500000e+07
Name: Price, dtype: float64

In [50]:
df['Sqft'].describe()

count     3221.000000
mean      2078.991618
std       1834.282270
min        205.000000
25%       1291.000000
50%       1760.000000
75%       2472.000000
max      76012.000000
Name: Sqft, dtype: float64

In [51]:
######Calculating Quartiles, IQR and removing the outliers########

In [52]:
Q1_Price = df['Price'].quantile(0.25)
Q3_Price = df['Price'].quantile(0.75)
IQR_Price = Q3_Price - Q1_Price
IQR_Price

995000.0

In [53]:
Q1_Beds = df['Beds'].quantile(0.25)
Q3_Beds = df['Beds'].quantile(0.75)
IQR_Beds = Q3_Beds - Q1_Beds
IQR_Beds

2.0

In [54]:
Q1_Bath = df['Bath'].quantile(0.25)
Q3_Bath = df['Bath'].quantile(0.75)
IQR_Bath = Q3_Bath - Q1_Bath
IQR_Bath

1.0

In [55]:
Q1_Sqft = df['Sqft'].quantile(0.25)
Q3_Sqft = df['Sqft'].quantile(0.75)
IQR_Sqft = Q3_Sqft - Q1_Sqft
IQR_Sqft

1181.0

In [56]:
df['Price'] = df['Price'][~((df['Price'] < (Q1_Price - 1.5 * IQR_Price)) |(df['Price'] > 
                                                                           (Q3_Price + 1.5 * IQR_Price)))]

/var/folders/lz/wgmsmvbx2_38594ymqn4b_f00000gn/T/ipykernel_68199/2998803243.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'] = df['Price'][~((df['Price'] < (Q1_Price - 1.5 * IQR_Price)) |(df['Price'] >


In [57]:
df['Beds'] = df['Beds'][~((df['Beds'] < (Q1_Beds - 1.5 * IQR_Beds)) |(df['Beds'] > 
                                                                           (Q3_Beds + 1.5 * IQR_Beds)))]

/var/folders/lz/wgmsmvbx2_38594ymqn4b_f00000gn/T/ipykernel_68199/1444743787.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Beds'] = df['Beds'][~((df['Beds'] < (Q1_Beds - 1.5 * IQR_Beds)) |(df['Beds'] >


In [58]:
df['Bath'] = df['Bath'][~((df['Bath'] < (Q1_Bath - 1.5 * IQR_Bath)) |(df['Bath'] > 
                                                                           (Q3_Bath + 1.5 * IQR_Bath)))]

/var/folders/lz/wgmsmvbx2_38594ymqn4b_f00000gn/T/ipykernel_68199/3371694092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Bath'] = df['Bath'][~((df['Bath'] < (Q1_Bath - 1.5 * IQR_Bath)) |(df['Bath'] >


In [59]:
df['Sqft'] = df['Sqft'][~((df['Sqft'] < (Q1_Sqft - 1.5 * IQR_Sqft)) |(df['Sqft'] > 
                                                                           (Q3_Sqft + 1.5 * IQR_Sqft)))]

/var/folders/lz/wgmsmvbx2_38594ymqn4b_f00000gn/T/ipykernel_68199/3064142063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sqft'] = df['Sqft'][~((df['Sqft'] < (Q1_Sqft - 1.5 * IQR_Sqft)) |(df['Sqft'] >


In [60]:
df.isna().sum() #number of outliers removed

Street             0
City               0
County             0
State              0
Zipcode            0
Price            248
WalkScore          0
Bike_Score         0
Beds              35
Bath             136
Sqft             147
Property_Type      0
Year_Built       213
Status             0
dtype: int64

In [61]:
df=df.dropna() #removing the null values. The outliers were removed and replaced by null values

In [62]:
df

,Street,City,County,State,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,655 12th st #112,Oakland,Alameda-County,Ca,94607,649000.0,96,91.0,2.0,2.0,1037.0,Condo,2003.0,New
1,4475 redding st,Oakland,Alameda-County,Ca,94619,699000.0,82,47.0,2.0,2.0,990.0,Single family residential,1930.0,New
2,6501 san pablo ave #404,Oakland,Alameda-County,Ca,94608,685000.0,94,86.0,1.0,1.5,1010.0,Condo,2006.0,New
3,29467 taylor ave,Hayward,Alameda-County,Ca,94544,899999.0,52,69.0,3.0,2.5,1469.0,Single family home,1989.0,Active
5,1258 144th ave,San leandro,Alameda-County,Ca,94578,499950.0,80,55.0,2.0,1.0,545.0,Single family residential,1920.0,New
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3246,7100 county road 49,Guinda,Yolo-County,Ca,95637,1185000.0,16,50.0,3.0,2.5,2435.0,Single family residential,1990.0,Active
3247,543 1st st,Woodland,Yolo-County,Ca,95695,925000.0,85,79.0,4.0,2.5,2560.0,Single family residential,1890.0,Active
3248,20900 county road 95,Woodland,Yolo-County,Ca,95695,2300000.0,0,25.0,3.0,2.0,1744.0,Single family residential,1960.0,Active
3249,15644 state highway 16,Capay,Yolo-County,Ca,95607,829000.0,0,48.0,3.0,2.5,2640.0,Mobile/manufactured home,2001.0,Active


In [63]:
#Exporting to CSV

In [64]:
df.to_csv('redfin.csv',index=False)